In [1]:
import numpy as np
import string
import itertools
import base64
import hashlib

In [2]:
A = list(string.ascii_uppercase) + [str(i) for i in range(9)] 
C = ['A','D','F','G','V','X']

In [3]:
class UnmatchedParametersException(Exception):
    pass

In [4]:
def chunks(l, n):
    ''' Return a matrix having n columns, the ith column is the ith chunk of l
        
    Parameters
    ----------
    l : list
        list of digits
    n : int 
        number of chuncks
        
    Raises
    ------
    UnmatchedParametersException 
        If the lenght of l is not a multiple of n
        
    Returns
    -------
    np.array
        matrix having n columns, the ith column is the ith chunk of l
    '''
    
    if (len(l)%n != 0):
        raise UnmatchedParametersException('The lenght of l must be a multiple of n')
        
    chunk_len = len(l)//n 
    return np.array([l[i:i+chunk_len] for i in range(0, len(l), chunk_len)]).T

In [5]:
def get_inverse_perm(perm):
    ''' Get the inverse permutation of perm
    
    Parameters
    ----------
    perm : list
        permutation to invert
    
    Returns
    -------
    list
        perm^(-1)
    '''
    
    perm_len = len(perm)
    inv_perm = np.zeros(perm_len, dtype=int)

    for i in range(perm_len):
        inv_perm[perm[i]] = i
        
    return inv_perm

In [6]:
def permute_columns(m, perm):
    ''' Permute m columns according to perm
    
    Parameters
    ----------
    m : np.array
        matrix to permute columns of
    perm : list
        list containing the new indices of m's columns
        
    Returns
    -------
    np.array
        matrix with permuted columns
    '''
    return m[:, perm]

In [7]:
def pad(s, n_columns, padding_char):
    ''' Write s on a matrix having n_columns columns and pad the last compiled row with padding_chars
        
    Parameters
    ----------
    s : str
        string to write in the matrix and then pad
    padding_char : str
        character used to pad s
        
    Returns 
    -------
    np.array
        matrix containing the padded string
    '''
    
    pad_lenght = (n_columns - len(s)%n_columns)%n_columns
    padded_s = s + padding_char*pad_lenght
    
    return np.array(list(padded_s)).reshape((-1, n_columns))    

In [8]:
def unpad(m, padding_char):
    ''' Read the text following the rows, then return the unpadded text
    Note that the text has always even lenght, so:
    - if n_columns is even, the number of padding digits is even 
    - if n_columns AND n_rows are odd, the number of padding digits is odd, 
    otherwise it is even
        
    Parameters
    ----------
    m : np.array
        matrix containing the text to unpad
    padding_char : str
        character used to pad initial string
        
    Returns 
    -------
    list 
        all the possible unpadded strings
    '''
    
    possible_results = []
    n_rows = m.shape[0]
    n_columns = m.shape[1]
    
    # Get the text in m in form of a string
    flat_inv_text = ''.join(m.flatten())

    # If the number of padding digits is odd, the last digit is for sure a padding one
    if(n_rows%2 != 0 and n_columns%2 != 0):
        flat_inv_text = flat_inv_text[:-1]
    
    while(True):
        # The obtained text is for sure one possible result
        possible_results.append(flat_inv_text)
        last_two_char = flat_inv_text[-2:]
        # If the text contains two padding char at the end 
        # Then we can remove them and obtain another valid result
        if(last_two_char != padding_char*2):
            break
        else:
            flat_inv_text = flat_inv_text[:-2]
            
    return possible_results

In [9]:
def build_substitution_dict(phi):
    ''' Build a dictionary that can be used to operate a substitution
    
    Parameters
    ----------
    phi : str
        flattened substitution matrix
    C : list
        list containing the alphabet of the ciphertext
        
    Raises 
    ------
    UnmatchedParametersException
        if the shape of the matrix obtained by phi is shaped differently from (len(C), len(C))
        
    Returns
    -------
    dict
        dictionary mathing every char in phi with its substitution in CxC
    '''
    
    
    alpha_size = len(C)
    
    if len(phi) != alpha_size**2:
        raise UnmatchedParametersException('Invalid substitution matrix')
    
    sub_dict = {}
    for i in range(len(phi)):
        # Don't add missing characters to the dictionary
        if (phi[i] != '\x00'):
            sub_dict[phi[i]] = C[i//alpha_size] + C[i%alpha_size]
        
    return sub_dict

In [10]:
def build_inversion_dict(phi):
    ''' Build a dictionary to invert the substitution phi having codomain C
    
    Parameters
    ----------
    phi : str
        string containing lines of the substitution matrix
    C : list
        list containing the alphabet of the ciphertext, 
        their indices are used to match characters in phi with a couple of characters in C
        
    Raises 
    ------
    UnmatchedParametersException
        If the shape of the matrix obtained by phi is shaped differently from (len(C), len(C))
        
    Returns
    -------
    dict
        Dictionary mathing every 2-char long string to its inverse subtiture
    '''
    
    alpha_size = len(C)
    
    if len(phi) != alpha_size**2:
        raise UnmatchedParametersException('Invalid substitution matrix')
    
    # Build a dictionary for the inverse substitution
    inv_dict = {}
    for i in range(alpha_size):
        for j in range(alpha_size):
            # Don't add missing characters to the dictionary
            if (phi[i] != '\x00'):
                inv_dict[C[i]+C[j]] = phi[i*alpha_size + j]
            
    return inv_dict

In [11]:
def apply_inv_substitution_plus(s, inv_sub_dict):
    ''' Invert the substitution on s by using the inverse substitution dictionary.
    The "plus" refers to the fact that special characters are taken into account
    
    Parameters
    ----------
    s : str
        string to which the invert substition has to be applied
    inv_sub_dict : dict
        dictionary used to match string chunks to their respective (inverse) substitution  
    
    Returns
    -------
    str
        String obtained by reverting s 
    '''

    curr_idx = 0
    
    s_prime = ''
    
    while(curr_idx < len(s)):
        # Analyse s by taking one chunk at a time of lenght 2
        curr_chunck = s[curr_idx:curr_idx+2] 
        if curr_chunck in inv_sub_dict.keys():
            # if any substitution exists for curr_chunk in inv_sub_dict append it to the resulting string,
            s_prime += inv_sub_dict[curr_chunck]
        else:
            # otherwise copy just one character of the chunk in the resulting string
            s_prime += curr_chunck[0]

        # Move the cursor 2 indices ahead
        curr_idx += 2   
        
    return s_prime

In [12]:
def apply_substitution_plus(s, sub_dict):
    ''' Apply the substitution sub_dict to s
    The "plus" refers to the fact that special characters are taken into account
    
    Parameters
    ----------
    s : str
        string to which the substition has to be applied
    sub_dict : dict
        dictionary used to match strings to their respective substitution  
    
    Returns
    -------
    str
        String obtained by operating the substitution sub_dict on s
    '''

    curr_idx = 0
    
    s_prime = ''
    
    while(curr_idx < len(s)):
        # Analyse s by taking one character at a time
        curr_char = s[curr_idx]         
        if curr_char in sub_dict.keys():
            # if any substitution exists for curr_char in sub_dict append it to the resulting string,
            s_prime += sub_dict[curr_char]
        else:
            # otherwise duplicate the character in the resulting string
            s_prime += curr_char*2
        curr_idx += 1  
        
    return s_prime

## Part A

In [13]:
def decrypt(ct, sk):
    ''' Decrypt the ciphertext given the secret key. 
    Note that more than one corresponding plaintexts are possible 
    (ambiguity due to the ill-defined reverse padding operation).
    
    Parameters
    ----------
    ct : str
        ciphertext
    sk: list
        secret key formatted as follows [phi, gamma, sigma] where:
            - phi is the flattened substitution matrix
            - gamma is the padding character
            - sigma is the permutation applied
    
    Returns
    -------
    list
        list containing all the possible corresponding plaintexts
    '''
    
    possible_pts = []
    
    phi = sk[0] # substitution matrix in form of a string
    gamma = sk[1] # padding character
    sigma = sk[2] # permutation
    
    n_columns = len(sigma)
    
    # Split ciphertext in len(sigma) chuncks
    columns = chunks(list(ct), n_columns)
    
    # Get the inverse permutation of sigma 
    inv_sigma = get_inverse_perm(sigma)
    
    # Sort the columns according to sigma
    ord_columns = permute_columns(columns, inv_sigma)
    
    # Get the possible unpadded strings 
    possible_unpadded_str = unpad(ord_columns, gamma)
    
    # Get the inverse substitution dictionary
    inv_sub_dict = build_inversion_dict(phi)
    
    for s in possible_unpadded_str:
        # For each unpadded string, operate inverse substitution and 
        # compare the lenght of the result with the correct lenght of the plaintext
        tmp_pt = apply_inv_substitution_plus(s, inv_sub_dict)
        possible_pts.append(tmp_pt)
    
    return possible_pts

In [14]:
# A - Test1

pt_test = 'GNU1'
ct_test = 'AGDVFXGXDXFX'
phi_test = ''.join(list(string.ascii_uppercase) + [str(i) for i in range(10)])
gamma_test = 'X'
sigma_test = [1,0,2,4,3,5]
pt_len_test = 4

possible_pts = decrypt(ct_test, [phi_test, gamma_test, sigma_test])
for p in possible_pts:
    if len(p) == pt_len_test:
        if p == pt_test:
            print('Correct')

Correct


In [15]:
# A - Test2

Q3a_x='ERLWLHAEENOSEFAN'
Q3a_y='DADXFGFGXDFFXGGAFFFGAGGDFGADAXVDDFG'
Q3a_S='MP8ZTQYC0E5OLAFXK3RIH749WBJ6U2SGNV1D'
Q3a_c='G'
Q3a_s=(0, 1, 4, 5, 3, 2, 6)
Q3a_n=16

possible_pts = decrypt(Q3a_y, [Q3a_S, Q3a_c, Q3a_s])
for p in possible_pts:
    if len(p) == Q3a_n:
        if p == Q3a_x:
            print('Correct')

Correct


In [16]:
# Personal parameters

Q3a_y="FXGFGGDAFFFFFGXVXGGGDAAGGDVFFFVXDDG"
Q3a_S='16LQVIKOCS70PAJZMN4BEWUX2G38RHF5D9TY'
Q3a_c="G"
Q3a_s=(2, 0, 1, 5, 6, 3, 4)
Q3a_n=16

possible_pts = decrypt(Q3a_y, [Q3a_S, Q3a_c, Q3a_s])
for p in possible_pts:
    if len(p) == Q3a_n:
        print(p)

EARONHFLESLEAWNE


 ## Part B

In [17]:
def encrypt_plus(pt, sk):
    ''' Encrypts the plaintext using the given secret key.
    The "plus" refers to the fact that special characters are taken into account.
  
    Parameters
    ----------
    pt : str
        plaintext
    sk: list
        secret key formatted as follows [phi, gamma, sigma] where:
            - phi is the flattened substitution matrix
            - gamma is the padding character
            - sigma is the permutation to apply
    
    Returns
    -------
    str
        ciphertext
    '''
    
    phi = sk[0] # substitution matrix in form of a string
    gamma = sk[1] # padding character
    sigma = sk[2] # permutation
    
    n_columns = len(sigma)
    
    # Create a substitution dictionary
    sub_dict = build_substitution_dict(phi)
    
    # First apply the substitution to the plaintext
    pt_sub = apply_substitution_plus(pt, sub_dict)
    
    # Then pad the string and write it on n_columns columns
    padded_matrix = pad(pt_sub, n_columns, gamma)
    
    # Permute columns
    ct_matrix = permute_columns(padded_matrix, sigma)
    
    ct = ''
    for i in range(n_columns):
        ct += ''.join(ct_matrix[:,i])
    return ct

In [18]:
# B - Test1

pt_test = 'GNU1'
phi_test = ''.join(list(string.ascii_uppercase) + [str(i) for i in range(10)])
gamma_test = 'X'
sigma_test = [1,0,2,4,3,5]
ct_test = 'AGDVFXGXDXFX'

if encrypt_plus(pt_test, [phi_test, gamma_test, sigma_test]) == ct_test:
    print('Correct')

Correct


In [19]:
# B - Test2

Q3a_x='ERLWLHAEENOSEFAN'
Q3a_S='MP8ZTQYC0E5OLAFXK3RIH749WBJ6U2SGNV1D'
Q3a_c='G'
Q3a_s=(0, 1, 4, 5, 3, 2, 6)
Q3a_y='DADXFGFGXDFFXGGAFFFGAGGDFGADAXVDDFG'

if encrypt_plus(Q3a_x, [Q3a_S, Q3a_c, Q3a_s]) == Q3a_y:
    print('Correct')

Correct


In [20]:
# B - Test3

Q3b_x="WELCOME TO THE BONFIRE, UNKINDLED ONE. I AM A FIRE KEEPER. I TEND TO THE FLAME, AND TEND TO THEE. THE LORDS HAVE LEFT THEIR THRONES, AND MUST BE DELIVER'D TO THEM. TO THIS END, I AM AT THY SIDE."
Q3b_S='FY9S0HAXLVP6G5DJMTCROZ14K2E8QINUWB37'
Q3b_c="X"
Q3b_s=(3, 6, 10, 9, 1, 4, 0, 11, 2, 5, 12, 8, 7)
Q3b_y="FF VXG VF XA XXAF XA,X G FGVXVGFG,FAFDVX FXFVF FVDDGV'AFV FFFXADV.D .AAV X.FDAGAF V FXFVA.FFAXF.  DXXFFFFDXAXXA F FFAFX.F FGA XAFV  ,V FDV FGFFVF X AADXXFXFDXD GADAAXA AX DFDFX X XXFVXVFV V  F,X  GG  A VGG.V,D VAAXF AV.FXV G.GAFDVFXXFVAF DXV  DV  AVXF  FFXFFFX,V F  A FGFGG,AXAGVFFDAFXVGDXG AF'XG   FVXVAF  A FVFFF FDX F GDX.X,  XG X DF FGFFAF V AFVFXXDFV XAAFA F FVVVFV DA F  VFGAFFFX FDXV"

if encrypt_plus(Q3b_x, [Q3b_S, Q3b_c, Q3b_s]) == Q3b_y:
    print('Correct')

Correct


In [21]:
# Personal parameters

Q3b_x="WELCOME TO THE BONFIRE, UNKINDLED ONE. I AM A FIRE KEEPER. I TEND TO THE FLAME, AND TEND TO THEE. THE LORDS HAVE LEFT THEIR THRONES, AND MUST BE DELIVER'D TO THEM. TO THIS END, I AM AT THY SIDE."
Q3b_S='LS0HQMXZ61YIOA7BGW5J82NU3KRT9FVCPDE4'
Q3b_c="D"
Q3b_s=(11, 8, 0, 5, 2, 4, 7, 1, 10, 6, 9, 3, 12)

encrypt_plus(Q3b_x, [Q3b_S, Q3b_c, Q3b_s])

"XAXGV DV.XGX F.FDVFXGFXVXAG VDF G AV VVVVDX X AVDAGGAGX GDAVFVXXVGD X  V,G  VA  A XAF.D,F AFG,VGDVFVAADXGXDAGV AG'GF   XX  FD  XXXV  VVGXVVG,X V  A VDAGFVGAFXX FXFVAGA AG XXFVG X XD A GXXXVX FV V  XVFDVVXG VFGXX VVD XVVD  ,X VFX VDAVXA X DAAGVXX.F .VAX G.AFXVVX D AGXXD.XAF,XVAFXG AGGXV AXFFDX'AAX VGAVVGA.  FGGAGVVAGVXGV A VVVAG.VV XXF DV GV GGAG GA,G V VDDGDXGDVX  D GXVVA AXG V GXG.G,  D"

## Part C 

In [22]:
P = string.punctuation + string.whitespace

In [23]:
def apply_incomplete_substitution(s, sub_dict):
    ''' Apply substitution on s by using the substitution dictionary sub_dict,
    if an entry is not present in the dictionary insert '__' instead.
    
    Parameters
    ----------
    s : str
        string to which the substition has to be applied
    sub_dict : dict
        dictionary used to match characters to their respective substitution  
    
    Returns
    -------
    str
        String obtained by applying the substitution to s 
    np.array.char
        Array containing the characted for which a substitution does not exist
    '''
    
    curr_idx = 0
    s_prime = ''
    missing_subs = np.char.array(list('_' * (len(s))))
    
    while(curr_idx < len(s)):
        # Analyse s by taking one character at a time
        curr_char = s[curr_idx] 
        if curr_char in sub_dict.keys():
            # if any substitution exists for curr_cunk in sub_dict append it to the resulting string
            s_prime += sub_dict[curr_char]
        elif curr_char in P:
            # if it is a punctuation symbol or whitespace then append a duplicate of it in the resulting string
            s_prime += curr_char*2
        else:
            # otherwise it is a missing entry in the substitution matrix, so append a doubled conventional special character
            # and add the char to the array of missing substitutions at index cur_idx
            s_prime += '_' * 2
            missing_subs[curr_idx] = curr_char
            
        curr_idx += 1  
        
    return s_prime, missing_subs

In [24]:
def fill_sub_dict(m_miss, m_compl, missing_subs, sub_dict):
    ''' Add entries to the incomplete substitution dictionary by reverse engineering 
    the missing ones. To do it we need to know a plaintext p and its corresponding ciphertext c.
    
    Parameters
    ----------
    m_miss : np.array
        matrix obtained by applying the incomplete substitution to p and then padding the resulting matrix
    m_compl : np.array
        matrix obtained by applying the inverse permutation to c
    missing_subs : np_array
        array of the same lenght of p, at each position i it contains:
            - a '_' if the character p[i] has a substitution in sub_dict
            - p[i] if a substitution for it is not available in sub_dict
    sub_dict : dict
        dictionary containing the available substitutions in the substitution matrix
        
    Raises
    ------
    UnmatchedParametersException
        if there is any incoherence between m_miss and m_compl or 
        if one tries to add two different values for the same key in the substitution dictionary
        
    Returns
    -------
    dict
        the substitution dictionary filled with all the new substitutions one could reverse
        enginner from the parameters
    '''
    m_miss_flat = m_miss.flatten()
    m_compl_flat = m_compl.flatten()
    for i in range(0, len(m_miss_flat) - 1, 2):
        curr_chunk = m_miss_flat[i] + m_miss_flat[i+1]
        actual_chunk = m_compl_flat[i] + m_compl_flat[i+1]
        if curr_chunk == '__':
            # if curr_chunk is a '__' then a substitution for it is missing in sigma
            missing_sub = missing_subs[i//2] # char that generated a '__' in the substitution matrix (its substitution is missing)
            if missing_sub not in sub_dict.keys():
                # if there is no entry in the substitution table for missing_sub add it
                sub_dict[missing_sub] = actual_chunk
            elif sub_dict[missing_sub] != actual_chunk:
                # if there is an entry in the substitution table for missing_sub 
                # and it is different from actual char the substitution does not match
                raise UnmatchedParametersException

        elif curr_chunk != actual_chunk:
            # if there is inconsistence in the two matrices than the substitution is wrong
            raise UnmatchedParametersException
            
    return sub_dict

In [25]:
def find_compl_sub_dict(pt, ct, sk):
    ''' Compute all the possible complete versions of the substitution dictionary
    coherent with the given parameters
    
    Parameters
    ----------
    pt : str
        plaintext
    ct : str
        ciphertext
    sk: list
        secret key formatted as follows [phi, gamma, sigma] where:
            - phi is the flattened substitution matrix
            - gamma is the padding character
            - sigma is the permutation
    
    Returns 
    -------
    list
        list of all possible complete versions of the substitution dictionary
    '''
    
    possible_compl_sub_dict = [] # list of posssible complete substitution dictionaries
    S_miss = sk[0] # substitution matrix in form of a string
    gamma = sk[1] # padding character
    sigma = sk[2] # permutation

    ### Sub + Pad the plaintext

    n_columns = len(sigma)
    # Create a substitution dictionary
    sub_dict_miss = build_substitution_dict(S_miss)
    # First apply the substitution to the plaintext, taking notes of the missing substitutions
    pt_sub_miss, missing_subs  = apply_incomplete_substitution(pt, sub_dict_miss)  
    # Then pad the string and write it on n_columns columns
    m_miss = pad(pt_sub_miss, n_columns, gamma)

    ### Undo columns permutation on the ciphertext

    # Split ciphertext in l chuncks
    columns = chunks(list(ct), n_columns)
    # Get the inverse permutation of sigma 
    inv_sigma = get_inverse_perm(sigma)
    # Sort the columns according to sigma
    m_compl = permute_columns(columns, inv_sigma)

    ### Reverse engineer a more complete version of the substitution dictionary 
    ### from the two matrices and the array of missing substitutions
    filled_sub_dict = fill_sub_dict(m_miss, m_compl, missing_subs, sub_dict_miss.copy()) 

    ### Finally encrypt the plaintext with the new complete substitution table

    # First apply the substitution to the plaintext
    pt_sub = apply_substitution_plus(pt, filled_sub_dict)  
    # Then pad the string and write it on n_columns columns
    padded_matrix = pad(pt_sub, n_columns, gamma)
    # Permute columns
    ct_matrix = permute_columns(padded_matrix, sigma)

    ct_guess = ''
    for i in range(n_columns):
        ct_guess += ''.join(ct_matrix[:,i])
        
    ### Check whether the computer ciphertext is equal to the given solution, 
    ### if it is the case then compute all the possible complete versions of the substitution dictionary
    if ct_guess == ct:
        unmatched_keys = [c for c in A if c not in filled_sub_dict.keys()]
        unmatched_values = [''.join(p) for p in itertools.product(C, repeat=2) if ''.join(p) not in filled_sub_dict.values()]
        # for each possible combination of unmatched_key and unmatched_value
        for uv in itertools.permutations(unmatched_values):
            # create an extension for the substitution dictionary
            dict_extension = dict(zip(unmatched_keys, uv))
            # add it to the list of posssible complete substitution dictionaries
            possible_compl_sub_dict.append(dict(filled_sub_dict, **dict_extension))
                
    return possible_compl_sub_dict

In [26]:
def find_possible_params(sigma_len_interval, pt, ct, S_miss, gamma):
    ''' Find all the possible combinations of permutations and substitution dictionaries
    coherent with the given parameters
    
    Parameters
    ----------
    sigma_len_interval : tuple
        interval of possible lenghts of the permutation tuple
    pt : str
        plaintext
    ct : str
        ciphertext
    S_miss : np.array
        flattened substitution matrix with missing entries
    gamma : str
        character used to pad the plaintext after the substitution
    
    Returns
    -------
    list 
        list of all possible combinations of permutations and substitutions dictionaries
    '''
    possible_parameters = []
    for l in range(sigma_len_interval[0], sigma_len_interval[1]):
        for p in itertools.permutations(range(l)):
            try:   
                possible_compl_sub_dictionaries = find_compl_sub_dict(pt, ct, [S_miss, gamma, p])
                for d in possible_compl_sub_dictionaries:
                    possible_parameters.append((p, d))
            except UnmatchedParametersException: 
                continue
    return possible_parameters

In [27]:
def invert_sub_dict(sub_dict):
    ''' Swap keys and values of the given dictionary sub_dict
    '''
    return dict((v,k) for k,v in sub_dict.items())

In [28]:
def decrypt_given_inv_dict(ct, inv_sub_dict, gamma, sigma):
    ''' Decrypt the ciphertext given the already inverted substitution dictionary, 
    the padding char and the permutation applied. 
    Note that more than one corresponding plaintexts are possible 
    (ambiguity due to the ill-defined reverse padding operation).
    
    Parameters
    ----------
    ct : str
        ciphertext
    inv_sub_dict : dict
        already inverted substitution dictionary, it can be directly used for inverse substitution
    gamma : str
        padding character
    sigma : tuple
        permutation applied to the plaintext
    
    Returns
    -------
    list
        list containing all the possible corresponding plaintexts
    '''
    possible_pts = []
    
    n_columns = len(sigma)
    
    # Split ciphertext in len(sigma) chuncks
    columns = chunks(list(ct), n_columns)
    
    # Get the inverse permutation of sigma 
    inv_sigma = get_inverse_perm(sigma)
    
    # Sort the columns according to sigma
    ord_columns = permute_columns(columns, inv_sigma)
    
    # Get the possible unpadded strings 
    possible_unpadded_str = unpad(ord_columns, gamma)
    
    for s in possible_unpadded_str:
        # For each unpadded string, operate inverse substitution and 
        # compare the lenght of the result with the correct lenght of the plaintext
        tmp_pt = apply_inv_substitution_plus(s, inv_sub_dict)
        # If the lenght is the same, the plaintext is a valid candidate
        possible_pts.append(tmp_pt)
    
    return possible_pts

In [29]:
# C - Test 1
Q3c_x0="OBI-WAN: IT'S OVER, ANAKIN! I HAVE THE HIGH GROUND! ANAKIN SKYWALKER: YOU UNDERESTIMATE MY POWER! OBI-WAN: DON'T TRY IT."
Q3c_y0="VF-AG:DDA XX, GFDV VAFV AXAVA FDGX FDDG AADXVV AG GXFXXVDD G GAX! XVAF: VVX FADDDD-DV V'VVGF,DVAV!  XXXXX XGXGVGV! GFDVADAFAX: VFGVVVADGFV F VAF!VF-AG:FD'DXV V.XVAF: X' DVV FDDG!D DG DV DX XVFF!DVAV VFADDF:FD FFXVVDGXXGAADVV DD-DV XG' DF X."
Q3c_y1="D VV DGXDX V FGFDX AF,GF FDA..XXFXDVGVVV V DV DXXDX VVXX VXXDXXXGVGXADDVAAXXA DG XXXFX DVV VXXXVGVXDDXXXXX VAFDV. AV VAFDDADGDVGGD FFX DV VFVVAV D DV FVVVVF VGV XVD, DVGFV X DADVDDGD DV DGXX VGFF G VX DDXXVXDGFGDDVAXXXDV DFX V DV DXXA VGFVVDGFFD,XX AFGF GXG G DFV,DVFXGVDV D DV DXXDX VVXXXDXDV G AXXVXVDXDVGXVAXVDDGXDVDVF G AXFXGFDVXDDVDV DV XXAXDGF VX...FDDG DG FV G  XDFF DV AF AF.. DVV V FFXX G AXGFDVDVAXDD. VGFF DVFDV VDF VADGVAFV VVDVF AXF GDGDVV D XDG DV VGXXX.XXA VGFFDV GXXVV, GXF AXDXFXAXA G AXGAXXDDA VFX VDG,AF GFXDDFVGVGX AF AXDVDDF GGVVVADXD GVAD D GDVDV GDV FF FXF G AXGFDX. VVXD FDXDF DV VVVVAFVVVAGFVD V VXFGVG V AXGFDVDVAXDD D GXXVVXX VDDXF VAF D VVFDV F FGAVAXX VAFDV D FAVX AXVGFAVXVXDXD... XGXXVV GDDVD DXXAGF, DVDAF.. AXV A GVADDVXX XDFXV VGFV.AXVVD ADAFF VXF FVVG XXXFXDFVXXXDVDVVVDDA V VFV AXFXGFDV DDVAXDVXVX VGGXX,GVVVXX XDXFVDGVAXX GADFVGVXDGDFDAF XA DX V XGAXFX VVXX XGVVXGFXX D AFVAVGXA V DVXFG VFV GXVVVVDVXX XDFF.AXDDF GVDX, AXGDGX FVX D VXD, A VDFVGXXDXX XDFXV VGFV V VD D DV FVDVD AVV G FXXVG G FG D DV VGXXX V AVADXX DVVAXGX AFV..V"
Q3c_S='W\x00\x00P\x00H\x003\x001I\x00\x00075\x00\x00\x004\x00M\x00G6\x00Z28\x009\x00B\x00\x00X'
Q3c_c="V"
Q3c_H="c50688adcb123041c61e51dc2fe52ddc"

# Find all possible combination of permutations of lenght 2<=l<=6 and substitution dictionaries
# coherent with the parameters
possible_params = find_possible_params((2,7), Q3c_x0, Q3c_y0, Q3c_S, Q3c_c)
for t in possible_params:
    perm = t[0]
    compl_sub_dict = t[1]
    inv_sub_dict = invert_sub_dict(compl_sub_dict)
    possible_pts = decrypt_given_inv_dict(Q3c_y1, inv_sub_dict, Q3c_c, perm)
    
    # For each candidate plaintext obtained, encode it in base64 and then 
    # compute the md5 digest; if it is the same of the given one, then the plaintext is correct one 
    for p in possible_pts:
        pt_b64 = base64.b64encode(p.encode())
        H = hashlib.md5(pt_b64).hexdigest()
        if H == Q3c_H:
            print(p)

A LONG TIME AGO IN A GALAXY FAR, FAR AWAY... THERE IS UNREST IN THE GALACTIC SENATE. SEVERAL THOUSAND SOLAR SYSTEMS HAVE DECLARED THEIR INTENTIONS TO LEAVE THE REPUBLIC. THIS SEPARATIST MOVEMENT, UNDER THE LEADERSHIP OF THE MYSTERIOUS COUNT DOOKU, HAS MADE IT DIFFICULT FOR THE LIMITED NUMBER OF JEDI KNIGHTS TO MAINTAIN PEACE AND ORDER IN THE GALAXY. SENATOR AMIDALA, THE FORMER QUEEN OF NABOO, IS RETURNING TO THE GALACTIC SENATE TO VOTE ON THE CRITICAL ISSUE OF CREATING AN ARMY OF THE REPUBLIC TO ASSIST THE OVERWHELMED JEDI....


In [30]:
# Personal parameters

Q3c_x0="OBI-WAN: IT'S OVER, ANAKIN! I HAVE THE HIGH GROUND! ANAKIN SKYWALKER: YOU UNDERESTIMATE MY POWER! OBI-WAN: DON'T TRY IT."
Q3c_y0="XGGF: G' DVF FVAD!G VG DV GX XFAX!VDDG GFXAAX:FD AXGVGGDGGAFXDVF DG-VD XD' DF G.DG-VD G'GFGX,VDDG!  XAGGX XAXAFFD! DFGDDAGFDG: FAFDVFDDAFV F FXX!FG-XD:XD'DGF G.FG-XD:GDD AG, DFGD GVFV VGVGV XDDX FVAD DFVAVF FF DXXGGGVD D AGG! XGGF: FDG XFGD"
Q3c_y1="GXFD GVFVGX VGGFAGF X VFV AFFGD VGGGV?G VDAX DG.GDD FDV GDF VGGGG FAX VA FF.GDD  GGXAGVD. VFV AFFGD VGV VF AXXFDGX GGX G AGGXA FX F GG X AFA AV VGXDVGGDGDAADA DV GXVXFFVD D AVFV GD. V VXDAV  AFXVXV X VGXFD GX DVDVGVDAXVGD VG DV DG X AXG FFA DFDXFDX DV VF GXGFDGX DXA G FXFVXF F VD FGAGGD FD ADGXG D G ADFFFA.VGXGVD G AGGXA FX VGFDF VGA V VGVDVG D XD FGDXVG AGG, VGV AGGAAA, X FADG X XX.FDFFFDGGF,VGGFAX XD XAVDGA GVFGXDXVGDDG,GXD GGVAXGGGV GAVXVG G XD AGX.GFDG. V FAX VG DVGD XD XVD, FDDD XAGADVXX FAVGX VF DV XFVX D XXD AVXVG DV GG?  DFAVDDD. G' DG  GFF DV VX XFA DAAFD. G' FDGV VXDX XXD AVXVG XD  XXAADX X VGDGV,DDXDVFFAVD G XDGVGVDAXFG DV FFV F DDFGVGGX DXGVAXGDGGDVFVD AXG X XX FA FDDGAXX D DV VF GXGGXG V FAX AG AVAGX DVGVGVFVXVGFDXFA FGA.GX XXADGV X VGXDVGGGV VDGF D DDFVGAGGG GAGVDDGVFGDXGFDVDXF. V VAAGDDXDVFFAVD DV DA DGD X XD XFGXFDGF ADGA GGXDVF XGA GVDFFAF D AFFV,VGXG. DDXDDFVA, V VAVDVG FXFDDVGVGXFVGA V DG, VG XD XAVDGA AAAX GDGDVG GVG. XDGA X AFA GAGFDVF DFDXGGX GG FDVGDGX.XG FF AG XVFGX DVXXFFDXFGXXAAAGGGX XDG GGXFXG FD G'GDD FDDXFGX FXGGDAXGGA FA G'GV GD AAGX.XFGXXAAAGGGF FXFD FF D DV GD, F FXXDA DXDDGGV V FAX DGGX DXA D GXAVDV VGAGGAADGFD F XGGGAGV.VGVF GVXV DDAGAGFDGX XXADGV VF X AFA GVDDGX VGFDD V VFX XDG XD XGD. VGXFD GX D DV FFV D  AGXVFGDAFF XGGDVGDDV FDGXX F V DDGAVA X GVFV F FXXDA DXGX DAFGXDXVGGF FXFX X VGADGD XD FXX,GXVXVGDDVAF,FDVDXG, V GX AXDGAVDAF X DFXG GGVAXGGGV AGFDGD X AVX DVDVG FXFDDVGDGAG XA D GGDAVA GFDV.VGVDAXDFV GXXGXFA VFV,XA DG GDVA."
Q3c_S='\x00\x000V7\x00K\x009S53U\x00\x00XQ82\x00\x00\x004\x00\x00Z\x00\x001\x00P\x00\x00B6\x00'
Q3c_c="V"
Q3c_H="2039f0a25d6702cf9a8e942faddd0362"

# Find all possible combination of permutations of lenght 2<=l<=6 and substitution dictionaries
# coherent with the parameters
possible_params = find_possible_params((2,7), Q3c_x0, Q3c_y0, Q3c_S, Q3c_c)
for t in possible_params:
    perm = t[0]
    compl_sub_dict = t[1]
    inv_sub_dict = invert_sub_dict(compl_sub_dict)
    possible_pts = decrypt_given_inv_dict(Q3c_y1, inv_sub_dict, Q3c_c, perm)
    
    # For each candidate plaintext obtained, encode it in base64 and then 
    # compute the md5 digest; if it is the same of the given one, then the plaintext is correct one 
    for p in possible_pts:
        pt_b64 = base64.b64encode(p.encode())
        H = hashlib.md5(pt_b64).hexdigest()
        if H == Q3c_H:
            print(p)

DID YOU EVER HEAR THE TRAGEDY OF DARTH PLAGUEIS THE WISE? I THOUGHT NOT. IT'S NOT A STORY THE JEDI WOULD TELL YOU. IT'S A SITH LEGEND. DARTH PLAGUEIS WAS A DARK LORD OF THE SITH, SO POWERFUL AND SO WISE HE COULD USE THE FORCE TO INFLUENCE THE MIDICHLORIANS TO CREATE LIFE. HE HAD SUCH A KNOWLEDGE OF THE DARK SIDE THAT HE COULD EVEN KEEP THE ONES HE CARED ABOUT FROM DYING. THE DARK SIDE OF THE FORCE IS A PATHWAY TO MANY ABILITIES SOME CONSIDER TO BE UNNATURAL. HE BECAME SO POWERFUL AND THE ONLY THING HE WAS AFRAID OF WAS LOSING HIS POWER, WHICH EVENTUALLY, OF COURSE, HE DID. UNFORTUNATELY, HE TAUGHT HIS APPRENTICE EVERYTHING HE KNEW, THEN HIS APPRENTICE KILLED HIM IN HIS SLEEP. IRONIC. HE COULD SAVE OTHERS FROM DEATH, BUT NOT HIMSELF.


## Part D

In [31]:
P = string.punctuation + string.whitespace

In [32]:
Q3d_x0="OBI-WAN: IT'S OVER, ANAKIN! I HAVE THE HIGH GROUND! ANAKIN SKYWALKER: YOU UNDERESTIMATE MY POWER! OBI-WAN: DON'T TRY IT."
Q3d_y0="A AG!F F!AG:XDXAAV' G VV!F A!GG:DVVVVX' VXV,GDDDXVXVDVV !GVVV:  XVV FXGV VAA VXGV:  VGX XDDF XVX GVDGVXGA AAG FDXAAXA  V-AFV XGF VG VV GV:VAG'FD VFXVDDAXADF-XV.V'VX AADXVXVVDGV-FA.AVD,AGGGVGDXXFG !GAF-GFX VDV XV XA GV:AGGAVVG GVV XGFGDVG  A"
Q3d_y1="FAFGD A D XD' G XXVFVV V V DX' G XVXD GXVDXVGXGDGGGXAVG A VAXVGAV V.FVGG F FVVAAVX X.AD  XA GVXX. VDFX FXDXAVV GFXDAGFXX GAAFAXG   VF AXV VVGVVDD   AX VXGVG VVXXFGGFAGDGG VVGVDGG GXVVVVAXDV  VV VAVF. XGVX"
Q3d_S='\x00\x00\x00\x00\x00\x00\x0043\x00\x00\x00\x00C\x005\x00\x001\x007\x00\x000\x002Z\x00B\x00P\x00\x00X\x008'
Q3d_c="X"
Q3d_s=(4, 5, 1, 3, 2, 8, 6, 10, 11, 0, 7, 9)
Q3d_H="aa5bd637f87f6c7e0ed64641995a0332"

# Find all possible combination of permutations and substitution dictionaries
# coherent with the parameters
possible_sub_dicts = find_compl_sub_dict(Q3d_x0, Q3d_y0, [Q3d_S, Q3d_c, Q3d_s])
for d in possible_sub_dicts:
    inv_sub_dict = invert_sub_dict(d)
    possible_pts = decrypt_given_inv_dict(Q3d_y1, inv_sub_dict, Q3d_c, Q3d_s)
    
    # For each candidate plaintext obtained, encode it in base64 and then 
    # compute the md5 digest; if it is the same of the given one, then the plaintext is correct one 
    for p in possible_pts:
        pt_b64 = base64.b64encode(p.encode())
        H = hashlib.md5(pt_b64).hexdigest()
        if H == Q3d_H:
            print(p)

ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTORS HAVE A WAY OF SEEING MORE OF OUR FAULTS THAN WE WOULD LIKE. IT'S THE ONLY WAY WE GROW.
ALL MENTOR

In [33]:
# Personal parameters

Q3d_x0="OBI-WAN: IT'S OVER, ANAKIN! I HAVE THE HIGH GROUND! ANAKIN SKYWALKER: YOU UNDERESTIMATE MY POWER! OBI-WAN: DON'T TRY IT."
Q3d_y0="X XA!A G!XG:GDGXXD' A'FD AGGDDDXGFGF-DA.-XAD GAF DA DA GX:AXA DD!X V!AD:FDADDG' D'FX VVFGDXDAXGF-AV.DGD,AFFFGDXAFFD !DDFGAGAX XXA VGDXFGX  AXDD,XAAADADGGFA !GXFX:  GDA AXGF AVV DGGX:  DAG DDGF GAF ADGDVADA ADD GGAAXAA  V-AXG AFF GD GV DX:VA"
Q3d_y1=",GAG?XFXAGXXDGFAGGXFXXGA X'D.XXXVGAG-VXVD A. AA  ADG-DAD FXD XGXG.,DDF?XFDDAXDGVXDADXVDDGV D'X.FAFGGXD-GDGD F.D! GGXD GVF! AGFAFAVA FAGDXVD X VVXXF GG DG.X! GDFG VAG! GDAXGDAG F FDA DGAVF DF AX  FGVA AXG AGGAF AF GD  GDAXDAAAAD X GGDDV GF AV. FX  DAF-GDG FDA DAAF."
Q3d_S='\x00\x00\x003\x00\x00\x00\x00Q\x006\x0070\x00J\x00\x00\x00\x00\x00\x00\x00X\x002Z\x0094\x00\x00\x0015\x00'
Q3d_c="V"
Q3d_s=(4, 10, 6, 5, 11, 1, 8, 0, 3, 2, 9, 7)
Q3d_H="a07400580b901cfadcf6609501a6ad30"

# Find all possible combination of permutations and substitution dictionaries
# coherent with the parameters
possible_sub_dicts = find_compl_sub_dict(Q3d_x0, Q3d_y0, [Q3d_S, Q3d_c, Q3d_s])
for d in possible_sub_dicts:
    inv_sub_dict = invert_sub_dict(d)
    possible_pts = decrypt_given_inv_dict(Q3d_y1, inv_sub_dict, Q3d_c, Q3d_s)
    
    # For each candidate plaintext obtained, encode it in base64 and then 
    # compute the md5 digest; if it is the same of the given one, then the plaintext is correct one 
    for p in possible_pts:
        pt_b64 = base64.b64encode(p.encode())
        H = hashlib.md5(pt_b64).hexdigest()
        if H == Q3d_H:
            print(p)

OH, MY! WHAT HAVE YOU DONE? I'M BACKWARDS. YOU FLEA-BITTEN FURBALL! ONLY AN OVERGROWN MOP-HEAD LIKE YOU WOULD BE STUPID ENOUGH TO...
